In [ ]:
# default_exp custom_tf_training

# Custom Tensorflow Training

> Extending tf.keras for custom training functionality

In [ ]:
#hide
from nbdev.showdoc import *
import tensorflow as tf
import fastcore

## Custom Loss Functions

## Loading and Saving Models with Custom Components

## Custom Activation Functions, Initializers, Regularizers and Constants

## Custom Metrics

## Custom Layers

## Custom Models

## Losses and Metrics Based on Model Internals

## Computing Gradients Using Autodiff

## Custom Training Loops